**Important note**! Before you turn in this lab notebook, make sure everything runs as expected:

- First, restart the kernel -- in the menubar, select Kernel → Restart.
- Then run all cells -- in the menubar, select Cell → Run All.

Make sure you fill in any place that says YOUR CODE HERE or "YOUR ANSWER HERE."

## UK Traffic Accidents

In this problem, you will work with and analyze some data about accidents in the UK from 2009 to 2011. This data was derived from Kaggle.  The original dataset can be found here: https://www.kaggle.com/daveianhickey/2000-16-traffic-flow-england-scotland-wales/data.

This problem has 4 exercises worth a total of 10 points.

## Setup

Run the following code cell, which will load the modules you'll need for this problem.

> **Note.** This problem involves SQLite and the `sqlite3` module. Since that module is not supported in Vocareum when using the Python 3.6 kernel, we have set this notebook to use Python 3.5. If you do any testing or prototyping on your local machine, keep in mind that you are still responsible for making your code work when submitted through the autograder on Vocareum, so be mindful of potential versioning differences.

In [1]:
import sys
import pandas as pd
import numpy as np
import sqlite3 as db

print("Python version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("Numpy version: {}".format(np.__version__))
print("SQLite3 version: {}".format(db.version))

from IPython.display import display
from cse6040utils import download_all, canonicalize_tibble, tibbles_are_equivalent

Python version: 3.6.3 |Anaconda, Inc.| (default, Nov  9 2017, 00:19:18) 
[GCC 7.2.0]
Pandas version: 0.23.4
Numpy version: 1.13.3
SQLite3 version: 2.6.0


## The Dataset

To help with your analysis, we will first drop any record that has missing value. We will also transform the column, `Date`, to have the structure yyyy-mm-dd.  Using this transformed Date column, we will then add a `Month` column to the dataset, which you will use in the exercises.

In [2]:
print('Loading dataset...This may take a while...')

datasets = {'accident_by_hour_soln.csv': '46ae91224473fc2d15794716d10231ce',
            'accidents_2009_to_2011.csv': '530ce2d51394f77a21fdd741a8ac9f0b',
            'max_dayofweek_soln.csv': '54f0f74c9ac05880e6a5b23d5d34f11b',
            'top10_location_soln.csv': '5b67bcf14fd719afe8444a00a3390c80'}
datapaths = download_all(datasets, suffix='accidents/')

#let's read the data into our environment
Accidents = pd.read_csv(datapaths["accidents_2009_to_2011.csv"])

#we will remove any rows that has missing values
Accidents = Accidents.dropna() 

#transform the Date column
Accidents['Date'] = pd.to_datetime(Accidents['Date'], dayfirst=True, infer_datetime_format=True).dt.date

#add the Month column
Accidents['Month'] = pd.to_datetime(Accidents['Date'], dayfirst=True, infer_datetime_format=True).dt.month


assert len(Accidents)==281765 # number of records
assert len(Accidents.columns) == 18 # number of columns

print('\nAfter preprocessing, Accidents has {} records and {} columns'.format(len(Accidents), len(Accidents.columns)))
print('\nFirst 5 records of Accidents')
Accidents.head()

Loading dataset...This may take a while...
Downloading: https://cse6040.gatech.edu/datasets/accidents/accident_by_hour_soln.csv ...


AssertionError: Downloaded file 'accidents/accident_by_hour_soln.csv' has incorrect checksum: 'a9a1e61648d3cb7ca839dfafa22e3c52' instead of '46ae91224473fc2d15794716d10231ce'

Let's take a look the column names for our dataset.

In [ ]:
print('\nA list of column names')
list(Accidents) # a list of column names

**Exercise 0** (2 points) Using the column, `Time`, which can be of the form HH:MM or H:MM, add a new column to the dataset called `Hour`.  We will use this new column in future exercises.


In [ ]:
#
# YOUR CODE HERE
#


In [ ]:
## Test Cell: exercise0 ##
assert len(Accidents['Hour'])== 281765
assert Accidents.iloc[0]['Hour']=='15'
assert Accidents.iloc[100]['Hour']=='7'
assert Accidents.iloc[1000]['Hour']=='12'
assert Accidents.iloc[10000]['Hour']=='13'
assert Accidents.iloc[100000]['Hour']=='15'
assert Accidents.iloc[200000]['Hour']=='14'
assert Accidents.iloc[281764]['Hour']=='18'

print("\n(Passed!)")

The following code cell creates an SQLite database file named `accident.db` and copies the Pandas dataframe that we had above into the database as a table named `Accidents`.

> For the exercises in this problem, you can either use the Pandas representation or the SQL representation, whichever helps you best solve the problem.

In [ ]:
# Import Accidents dataframe to sqlite database
# Connect to a database (or create one if it doesn't exist)

conn = db.connect('accident.db')
Accidents.to_sql('Accidents', conn, if_exists='replace', index=False)

Using SQL we can see the first 5 records of `Accidents`.

In [ ]:
pd.read_sql_query('SELECT * FROM Accidents LIMIT 5', conn)

**Exercise 1** (1 point) Determine the number of accidents that occur for each hour of the day.  Order the number of accidents in descending order. Save your result in a table named **`accident_by_hour`** with the columns named **`Hour`** and **`Num_of_Accidents`**, which is the number of accidents during that hour. 

In [ ]:
#
# YOUR CODE HERE
#

# Show your solution:
display(accident_by_hour)

In [ ]:
## Test Cell: exercise1 ##
# Read what we believe is the exact result
accident_by_hour_soln = pd.read_csv(datapaths['accident_by_hour_soln.csv'])

# Check that we got a data frame of the expected shape:
assert 'accident_by_hour' in globals(), "You need to store your results in a dataframe named `accident_by_hour`."
assert type(accident_by_hour) is type(pd.DataFrame()), "`accident_by_hour` does not appear to be a Pandas dataframe."
assert len(accident_by_hour) == len(accident_by_hour_soln), "The number of rows of `accident_by_hour` does not match our solution."
assert set(accident_by_hour.columns) == set(['Hour', 'Num_of_Accidents']), "Your table does not have the right set of columns."

assert tibbles_are_equivalent(accident_by_hour.astype('int64'), accident_by_hour_soln)
print("\n(Passed!)")

**Exercise 2** (3 points) Suppose we are interested in determining which day of the week had the most accidents in a particular year-month pair.

For each year and month, report the day of the week that had the largest number of accidents. Your result should be in ascending order by years then months, i.e., 2009-2011 for the year and 1-12 for the month. Save your result in a table called **`max_dayofweek`**.

Your table should contain the following columns: {`'Year'`, `'Month'`, `'Day_of_Week'`, `'Num_of_Accidents'`}.

For example, a row of this table might be `{2009, 1, 6, XXXX}`, where `XXXX` is the number of accidents observed in January 2009 on Friday. (In this data, days of the week are numbered starting at Sunday equals one.)

In [ ]:
#
# YOUR CODE HERE
#

# Show your solution:
display(max_dayofweek)

In [ ]:
## Test Cell: exercise2 ##
# Read what we believe is the exact result
max_dayofweek_soln = pd.read_csv(datapaths['max_dayofweek_soln.csv'])

# Check that we got a data frame of the expected shape:
assert 'max_dayofweek' in globals(), "You need to store your results in a dataframe named `max_dayofweek`."
assert type(max_dayofweek) is type(pd.DataFrame()), "`max_dayofweek` does not appear to be a Pandas dataframe."
assert len(max_dayofweek) == len(max_dayofweek_soln), "The number of rows of `max_dayofweek` does not match our solution."
assert set(max_dayofweek.columns) == set(['Year', 'Month', 'Day_of_Week', 'Num_of_Accidents']), "Your table does not have the right set of columns."

assert tibbles_are_equivalent(max_dayofweek, max_dayofweek_soln)
print("\n(Passed!)")

**Exercise 3** (4 points). Find the top 9 locations that had the most accidents. Report the **`Road_Surface_Conditions`** and the count of accidents under that condition. Store your result in table, **`top9_locations`**, which should contain the following columns:

{`'Location_Easting_OSGR'`, `'Location_Northing_OSGR'`, `'Road_Surface_Conditions'`, `'Num_of_Accidents'`}

> **Note.** We define a location by (`Location_Easting_OSGR`, `Location_Northing_OSGR`), i.e., Local British coordinates x-value, Local British coordinates y-value.

In [ ]:
#
# YOUR CODE HERE
#

# Show your solution:
display(top9_locations)

In [ ]:
## Test Cell: exercise3 ##
# Read what we believe is the exact result
top9_locations_soln = pd.read_csv(datapaths['top10_location_soln.csv'])

# Check that we got a data frame of the expected shape:
assert 'top9_locations' in globals(), "You need to store your results in a dataframe named `top9_locations`."
assert type(top9_locations) is type(pd.DataFrame()), "`top9_locations` does not appear to be a Pandas dataframe."
assert len(top9_locations) == len(top9_locations_soln), "The number of rows of `top9_locations` does not match our solution."
assert set(top9_locations.columns) == set(['Location_Easting_OSGR', 'Location_Northing_OSGR', 'Road_Surface_Conditions', 'Num_of_Accidents']), "Your table does not have the right set of columns."

assert tibbles_are_equivalent(top9_locations, top9_locations_soln)
print("\n(Passed!)")


In [ ]:
# Some cleanup code
conn.close()

** Fin ** You've reached the end of this problem. Don't forget to restart the kernel and run the entire notebook from top-to-bottom to make sure you did everything correctly. If that is working, try submitting this problem. (Recall that you *must* submit and pass the autograder to get credit for your work.)